In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import tensorflow as tf

import numpy as np
import libspn as spn

from deepsm.graphspn.spn_model import SpnModel, mod_compute_graph_up
from deepsm.graphspn.tbm.spn_template import TemplateSpn, NodeTemplateSpn, EdgeRelationTemplateSpn
from deepsm.graphspn.tbm.spn_instance import NodeTemplateInstanceSpn
from deepsm.graphspn.tbm.template import ThreeRelTemplate, SingleRelTemplate, SingleTemplate, RelTemplate
from deepsm.graphspn.tbm.graph_builder import build_graph
from deepsm.util import CategoryManager

# Load a trained view template SPN

In [61]:
spn_path = "/home/zkytony/Documents/thesis/experiments/deep-semantic-mapping/deepsm/experiments/results/graphspn/EdgeRelationTemplateExperiment/models/RelTemplate_2_Stockholm456.spn"

In [62]:
sess = tf.Session()

In [63]:
template_spn = EdgeRelationTemplateSpn(RelTemplate, seed=100)
template_spn.load(spn_path, sess)
SpnModel.print_structure(template_spn._root, sess)
print(template_spn._conc_inputs)
print(template_spn._view_dist_input)

[INFO] [spn.JSONLoader:load] Loading SPN graph from file '/home/zkytony/Documents/thesis/experiments/deep-semantic-mapping/deepsm/experiments/results/graphspn/EdgeRelationTemplateExperiment/models/RelTemplate_2_Stockholm456.spn'


Generating SPN structure...
[Using seed 100]
Node_22 (Sum)
    Input(Products1/Product1_10, None) (Product)
    Input(Products1/Product2_10, None) (Product)
    Input(Products1/Product3_9, None) (Product)
    Input(Products1/Product4_9, None) (Product)
    Input(Products1/Product5_9, None) (Product)
Products1/Product1_10 (Product)
    Input(Conc_EdgeRelationTemplateSpn_0_1_21, [0]) (Concat)
Products1/Product2_10 (Product)
    Input(Conc_EdgeRelationTemplateSpn_0_1_21, [1]) (Concat)
Products1/Product3_9 (Product)
    Input(Conc_EdgeRelationTemplateSpn_0_1_21, [2]) (Concat)
Products1/Product4_9 (Product)
    Input(Conc_EdgeRelationTemplateSpn_0_1_21, [3]) (Concat)
Products1/Product5_9 (Product)
    Input(Conc_EdgeRelationTemplateSpn_0_1_21, [4]) (Concat)
Conc_EdgeRelationTemplateSpn_0_1_21 (Concat)
    Input(View_EdgeRelationTemplateSpn_0_1_21, None) (IVs)
Conc_EdgeRelationTemplateSpn_0_1_21
View_EdgeRelationTemplateSpn_0_1_21


In [64]:
print(template_spn._conc_inputs)
print(template_spn._view_dist_input)

Conc_EdgeRelationTemplateSpn_0_1_21
View_EdgeRelationTemplateSpn_0_1_21


In [65]:
spn.display_spn_graph(template_spn._root, skip_params=False)

In [66]:
template_spn._root.is_valid()

True

In [67]:
template_spn.template

deepsm.graphspn.tbm.template.RelTemplate

In [68]:
SpnModel.print_structure(template_spn._root, sess)

Node_22 (Sum)
    Input(Products1/Product1_10, None) (Product)
    Input(Products1/Product2_10, None) (Product)
    Input(Products1/Product3_9, None) (Product)
    Input(Products1/Product4_9, None) (Product)
    Input(Products1/Product5_9, None) (Product)
Products1/Product1_10 (Product)
    Input(Conc_EdgeRelationTemplateSpn_0_1_21, [0]) (Concat)
Products1/Product2_10 (Product)
    Input(Conc_EdgeRelationTemplateSpn_0_1_21, [1]) (Concat)
Products1/Product3_9 (Product)
    Input(Conc_EdgeRelationTemplateSpn_0_1_21, [2]) (Concat)
Products1/Product4_9 (Product)
    Input(Conc_EdgeRelationTemplateSpn_0_1_21, [3]) (Concat)
Products1/Product5_9 (Product)
    Input(Conc_EdgeRelationTemplateSpn_0_1_21, [4]) (Concat)
Conc_EdgeRelationTemplateSpn_0_1_21 (Concat)
    Input(View_EdgeRelationTemplateSpn_0_1_21, None) (IVs)


In [69]:
SpnModel.print_weights(template_spn._root, sess)

Node_22 (Sum)
[ 0.0035598   0.34087345  0.29326344  0.24100113  0.12130217] [(5,)]


In [70]:
template_spn._conc_inputs.set_inputs()
print(template_spn._conc_inputs.inputs)
print(template_spn._num_nodes)
print(template_spn._conc_inputs)
print(template_spn._view_dist_input)

()
0
Conc_EdgeRelationTemplateSpn_0_1_21
View_EdgeRelationTemplateSpn_0_1_21


In [71]:
catg_inputs = spn.IVs(num_vars=6, num_vals=CategoryManager.NUM_CATEGORIES, name="Catg")
view_inputs = spn.IVs(num_vars=2, num_vals=5, name="View")
conc = spn.Concat(catg_inputs, view_inputs)
copied_tspn_root = mod_compute_graph_up(template_spn._root,
                                        TemplateSpn.dup_fun_up,
                                        tmpl_num_vars=[0, 1],
                                        tmpl_num_vals=[CategoryManager.NUM_CATEGORIES, 5],
                                        graph_num_vars=[catg_inputs.num_vars, view_inputs.num_vars],
                                        conc=conc,
                                        labels=[[],[0]])

In [72]:
SpnModel.print_weights(copied_tspn_root, sess)

Sum (Sum)
[ 0.0035598   0.34087345  0.29326344  0.24100113  0.12130217] [(5,)]


In [73]:
SpnModel.print_structure(copied_tspn_root, sess)

Sum (Sum)
    Input(Product_4, None) (Product)
    Input(Product_3, None) (Product)
    Input(Product_2, None) (Product)
    Input(Product_1, None) (Product)
    Input(Product, None) (Product)
Product_4 (Product)
    Input(Concat_1, [12]) (Concat)
Product_3 (Product)
    Input(Concat_1, [13]) (Concat)
Product_2 (Product)
    Input(Concat_1, [14]) (Concat)
Product_1 (Product)
    Input(Concat_1, [15]) (Concat)
Product (Product)
    Input(Concat_1, [16]) (Concat)
Concat_1 (Concat)
    Input(Catg_1, None) (IVs)
    Input(View_1, None) (IVs)


In [74]:
copied_tspn_root.is_valid()

True